In [76]:
from pycaret.datasets import get_data
#data = get_data('insurance')
import pandas as pd
import numpy as np
import os
#data = get_data('diamond')
path = os.getcwd()
parent_path=os.path.abspath(os.path.join(path, os.pardir))
data = pd.read_csv(parent_path+'/data/raw/insurance.csv')
print(data.head(5))

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520



Pycaret setup data preprocessing

In [77]:
from pycaret.regression import *
data = setup(data, target = 'charges', session_id = 123,
           normalize = True, log_experiment = True, experiment_name = 'MLOps_v1', log_data = True,
           polynomial_features = True, trigonometry_features = True, feature_interaction=True, 
           bin_numeric_features= ['age', 'bmi'])

,Description,Value
0,session_id,123
1,Target,charges
2,Original Data,"(1338, 7)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(936, 56)"


To get preprocessed dataset
X
y
X_train
X_test
y_train
y_test
seed
prep_pipe
experiment__

In [58]:
def save_csv_file(df,path):
    df.to_csv(path)

In [59]:
x_training_data = get_config(variable="X_train")
y_training_data = get_config(variable="y_train")

data_path=parent_path+'\data\processed\X_train.csv'
save_csv_file(training_data,data_path)

data_path=parent_path+'\data\processed\y_train.csv'
save_csv_file(training_data,data_path)
#print(training_data)

In [61]:
print(get_config)

<function get_config at 0x0000020448E25158>


In [62]:
best_model = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2788.7220,24600842.8718,4945.0543,0.8228,0.4674,0.3403,0.2550
llar,Lasso Least Angle Regression,3204.6926,25896000.3012,5074.0714,0.8173,0.4747,0.3437,0.0240
omp,Orthogonal Matching Pursuit,3167.1362,26343745.1791,5110.6374,0.8158,0.4267,0.3217,0.0160
br,Bayesian Ridge,3238.8572,26204585.3061,5105.7267,0.8145,0.4780,0.3471,0.0200
ridge,Ridge Regression,3235.1532,26287988.8000,5113.7899,0.8139,0.4712,0.3464,0.0240
lasso,Lasso Regression,3230.7756,26419158.8000,5126.5706,0.8130,0.4631,0.3453,0.0310
lr,Linear Regression,3240.3398,26492972.0000,5133.9737,0.8125,0.4615,0.3467,1.4690
lightgbm,Light Gradient Boosting Machine,3082.8000,26628035.3921,5147.6712,0.8074,0.5715,0.3866,0.2390
rf,Random Forest Regressor,2843.8886,26714181.6276,5152.6132,0.8074,0.4908,0.3583,0.7590
par,Passive Aggressive Regressor,2187.3929,28734763.5042,5332.9326,0.7994,0.3992,0.0864,0.1640


In [63]:
tuned_best = tune_model(best_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3305.5319,29162292.2686,5400.2122,0.8258,0.6846,0.4335
1,3561.7180,35744801.9213,5978.6957,0.7922,0.4929,0.3745
2,3510.0356,33444712.3137,5783.1404,0.5765,0.8129,0.5273
3,3559.9520,31267262.3349,5591.7137,0.7389,0.6518,0.5221
4,3700.8282,39003497.5678,6245.2780,0.7099,0.6955,0.4522
5,3212.9380,26275235.6301,5125.9375,0.8310,0.4771,0.3726
6,3085.7952,27279652.7760,5222.9927,0.8144,0.6231,0.3875
7,3183.2599,29752466.7399,5454.5822,0.8364,0.6046,0.3661
8,3009.6253,23288265.1034,4825.7917,0.8496,0.5734,0.4773


In [65]:
final_model = finalize_model(estimator = tuned_best) 

In [66]:
evaluate_model(final_model)

,Parameters
alpha,0.9
ccp_alpha,0.0
criterion,friedman_mse
init,None
learning_rate,0.15
loss,ls
max_depth,7
max_features,1.0
max_leaf_nodes,None
min_impurity_decrease,0.02


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [68]:
model_path=parent_path+'\models\deployment_gbr'
save_model(final_model, model_path)

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='charges',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy...
                                            learning_rate=0.15, loss='ls',
                                            max_depth=7, max_features=1.0,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.02,
                          

In [71]:
# create API
create_api(final_model, 'mlops_api_v1')


API sucessfully created. This function only creates a POST API, it doesn't run it automatically.

To run your API, please run this command --> !python mlops_api_v1.py
    


In [70]:
#!python mlops_api_v1.py

^C


In [72]:
deployment_gbr_model = load_model(model_path)

Transformation Pipeline and Model Successfully Loaded


In [73]:
print(deployment_gbr_model)

Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='charges',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy...
                                           learning_rate=0.15, loss='ls',
                                           max_depth=7, max_features=1.0,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.02,
                                          

In [81]:
#predictions = predict_model(deployment_gbr_model, data=x_test_data)

predictions = predict_model(deployment_gbr_model)
predictions.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,515.1714,827555.1049,909.7006,0.9942,0.1026,0.0656


,age_Power2,bmi_Power2,sex_female,children_0,children_1,children_2,children_3,children_4,children_5,smoker_no,...,bmi_Power2_multiply_age_Power2,smoker_no_multiply_children_0,bmi_5.0_multiply_age_Power2,region_southeast_multiply_bmi_Power2,smoker_no_multiply_age_Power2,region_southwest_multiply_bmi_Power2,region_northwest_multiply_age_Power2,sex_female_multiply_bmi_Power2,charges,Label
0,0.597312,2.259463,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.349604,0.0,0.0,2.259463,0.597312,0.000000,0.000000,2.259463,9800.888672,9171.534539
1,-0.624317,1.120715,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.699682,0.0,-0.0,0.000000,-0.624317,0.000000,-0.000000,0.000000,4667.607422,4478.455242
2,-0.886031,0.034522,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.030588,0.0,-0.0,0.000000,-0.000000,0.034522,-0.000000,0.034522,34838.871094,34018.218987
3,-0.445997,-1.040384,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.464008,0.0,-0.0,-0.000000,-0.445997,-0.000000,-0.445997,-0.000000,5125.215820,5207.540458
4,1.661025,-0.826600,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.373003,1.0,0.0,-0.826600,1.661025,-0.000000,0.000000,-0.000000,12142.578125,13212.331234


In [82]:
# create web application for testing using Gradio
create_app(deployment_gbr_model)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
